![title](maxresdefault.jpg)

In [1]:
import random, pylab

# set line width
pylab.rcParams['lines.linewidth'] = 4
# set font size for titles
pylab.rcParams['axes.titlesize'] = 20
# set font size for labels on axes
pylab.rcParams['axes.labelsize'] = 20
# set size of numbers on x-axis
pylab.rcParams['xtick.labelsize'] = 16
# set size of numbers on y-axis
pylab.rcParams['ytick.labelsize'] = 16
# set size of ticks on x-axis
pylab.rcParams['xtick.major.size'] = 7
# set size of ticks on y-axis
pylab.rcParams['ytick.major.size'] = 7
# set size of markers, e.g., circles representing points
# set numpoints for legend
pylab.rcParams['legend.numpoints'] = 1


In [2]:
class FairRoulette():
    def __init__(self):
        self.pockets = []
        self.name = 'Fair Roulette'
        for i in range(1, 37):
            self.pockets.append(i)
        self.ball = None
        self.pocketOdds = len(self.pockets) - 1

    def spin(self):
        self.ball = random.choice(self.pockets)

    def betPocket(self, pocket, amt):
        if str(pocket) == str(self.ball):
            return amt * self.pocketOdds
        else:
            return -amt

    def __str__(self):
        return 'Fair Roulette'

In [3]:
def playRoulette(game, numSpins, pocket, bet, toPrint):
    totPocket = 0
    for i in range(numSpins):
        game.spin()
        totPocket += game.betPocket(_pocket, bet)
    if toPrint:
        print(numSpins, 'spins of', game.name)
        print('Expected return betting', pocket, '=', \
              str(100 * totPocket / numSpins) + '%\n')
    return (totPocket / numSpins)

random.seed(0)
game = FairRoulette()
for numSpins in (100, 10000000):
    for i in range(3):
        playRoulette(game, numSpins, 2, 1, True)


100 spins of Fair Roulette
Expected return betting 2 = -100.0%

100 spins of Fair Roulette
Expected return betting 2 = 44.0%

100 spins of Fair Roulette
Expected return betting 2 = -28.0%

10000000 spins of Fair Roulette
Expected return betting 2 = 0.24596%

10000000 spins of Fair Roulette
Expected return betting 2 = -0.11548%

10000000 spins of Fair Roulette
Expected return betting 2 = -0.01756%



![title](roulette_wheel_1024x1024.jpg)

![title](double-zero-roulette.jpg)

## Law of large numbers
* 100 spins are not conclusive 
* 1000,000 has less variance
* The average of the results obtained from a large number of trials should be close to the expected value, and will tend to become closer as more trials are performed.



# Regression to the mean
* Following an extreme random event, the next random event is likely to be less extreme 
* If you spin a fair roulette wheel 10 times and get 100% reds, that is an extreme event (probability = 1/1024) 
* It is likely that in the next 10 spins, you will get fewer than 10 reds 
* But the expected number is only 5 
* So, if you look at the average of the 20 spins, it will be closer to the expected mean of 50% reds than to the 100% of the first 10 spins 


# Important concepts 
* The larger the sample size is, the closer you get to the mean of the population
* Resampling 


# Unfair Roulette


In [4]:
class EuRoulette(FairRoulette):
    def __init__(self):
        FairRoulette.__init__(self)
        self.pockets.append('0')
        self.name = 'EuRoulette'

    def __str__(self):
        return 'European Roulette'


In [5]:
class AmRoulette(EuRoulette):
    def __init__(self):
        EuRoulette.__init__(self)
        self.pockets.append('00')
        self.name = 'AmRoulette'

    def __str__(self):
        return 'American Roulette'


In [6]:
def findPocketReturn(game, numTrials, trialSize, toPrint):
    pocketReturns = []
    for t in range(numTrials):
        trialVals = playRoulette(game, trialSize, 2, 1, toPrint)
        pocketReturns.append(trialVals)
    return pocketReturns

In [7]:
def getMeanAndStd(X):
    mean = sum(X) / float(len(X))
    tot = 0.0
    for x in X:
        tot += (x - mean) ** 2
    std = (tot / len(X)) ** 0.5
    return mean, std

In [8]:
random.seed(0)
numTrials = 20
resultDict = {}
games = (FairRoulette, EuRoulette, AmRoulette)
for G in games:
    resultDict[G().__str__()] = []
for numSpins in (1000, 10000, 100000, 1000000):
    print('\nSimulate', numTrials, 'trials of',
          numSpins, 'spins each')
    for G in games:
        pocketReturns = findPocketReturn(G(), numTrials,
                                         numSpins, False)
        expReturn = 100 * sum(pocketReturns) / len(pocketReturns)
        print('Exp. return for', G().name, '=',
              str(round(expReturn, 4)) + '%')



Simulate 20 trials of 1000 spins each
Exp. return for Fair Roulette = 6.56%
Exp. return for EuRoulette = -2.26%
Exp. return for AmRoulette = -8.92%

Simulate 20 trials of 10000 spins each
Exp. return for Fair Roulette = -1.234%
Exp. return for EuRoulette = -4.168%
Exp. return for AmRoulette = -5.752%

Simulate 20 trials of 100000 spins each
Exp. return for Fair Roulette = 0.8144%
Exp. return for EuRoulette = -2.6506%
Exp. return for AmRoulette = -5.113%

Simulate 20 trials of 1000000 spins each
Exp. return for Fair Roulette = -0.0723%
Exp. return for EuRoulette = -2.7329%
Exp. return for AmRoulette = -5.212%


# Confidence Levels and Intervals

* Instead of estimating an unknown parameter by a single value (e.g., the mean of a set of trials), a confidence interval provides a range that is likely to contain the unknown value and a confidence that the unknown value lays within that range


* "The return on betting a pocket 10K time in European roulette is -3.3%. The margin of error is +/- 3.5% with a 95% level of confidence."
* What does this mean?
* If I were to conduct an infinite number of trials of 10k bets each,
    * My expected average return would be -3.3%
    * My return would be between roughly -6.8% and +0.2% 95% of the time


## Empirical Rule
Under some assumptions discussed later

* ~68% of data within one standard deviation of mean

* ~95% of data within 1.96 standard deviations of mean 

* ~99.7% of data within 3 standard deviations of mean

In [11]:
resultDict = {}
games = (FairRoulette, EuRoulette, AmRoulette) 
for G in games:
    resultDict[G().__str__()] = [] 
for numSpins in (100, 1000, 10000, 1000000, 100000000):
    print('\nSimulate betting a pocket for', numTrials, 'trials of', numSpins, 'spins each')
    for G in games:
        pocketReturns = findPocketReturn(G(), 20,numSpins, False) 
        mean, std = getMeanAndStd(pocketReturns)
        resultDict[G().__str__()].append((numSpins, 100*mean, 100*std))
        print('Exp. return for', G(), '=', str(round(100*mean, 3)) + '%,', '+/- ' + str(round(100*1.96*std, 3)) + '% with 95% confidence')


Simulate betting a pocket for 20 trials of 100 spins each
Exp. return for Fair Roulette = -17.2%, +/- 126.419% with 95% confidence
Exp. return for European Roulette = -4.6%, +/- 78.175% with 95% confidence
Exp. return for American Roulette = 8.0%, +/- 154.589% with 95% confidence

Simulate betting a pocket for 20 trials of 1000 spins each
Exp. return for Fair Roulette = -3.52%, +/- 45.815% with 95% confidence
Exp. return for European Roulette = 0.08%, +/- 37.109% with 95% confidence
Exp. return for American Roulette = -1.18%, +/- 34.673% with 95% confidence

Simulate betting a pocket for 20 trials of 10000 spins each
Exp. return for Fair Roulette = 0.008%, +/- 10.765% with 95% confidence
Exp. return for European Roulette = -3.772%, +/- 11.268% with 95% confidence
Exp. return for American Roulette = -6.274%, +/- 10.194% with 95% confidence

Simulate betting a pocket for 20 trials of 1000000 spins each
Exp. return for Fair Roulette = 0.086%, +/- 1.068% with 95% confidence
Exp. return fo